In [1]:
import pandas as pd
import numpy as np

def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

In [ ]:
data = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx")
display_full(data.head(2))

In [3]:
metadata = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/Finalized_dataset_1805_exams_with_Age.xlsx")
print(metadata.columns)
metadata.head(2)

Index(['Unnamed: 0', 'directory', 'source_directory', 'patient_directory',
       'PET-CT_info', 'system', 'npr', 'scan_date', 'dicom_img', 'patient_id',
       'patient_age', 'patient_weight', 'patient_size', 'imgsz_x', 'imgsz_y',
       'num_slices', 'voxsz_x', 'voxsz_y', 'slice_thickness', 'image_size',
       'voxel_size', 'img_pos', 'img_orient', 'for_uid', 'att_corr',
       'recons_method', 'image_type', 'aquisition_dt', 'aquisition_time',
       'study_desc', 'series_desc', 'protocol', 'corr_img', 'modality',
       'manufacturer', 'manufacturer_model', 'radiopharmaceutical',
       'radiopharmaceutical_volume', 'radiopharmaceutical_start_time',
       'radionuclide_total_dose', 'radionuclide_half_life',
       'radionuclide_positron_fraction',
       'radiopharmaceutical_start_date_time'],
      dtype='object')


,Unnamed: 0,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,dicom_img,patient_id,...,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_...,ASPTCTX0001,lpr385705046400,20140313,1.2.840.113619.2.55.3.4240671514.332.139417595...,lpr385705046400,...,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D...,ASPTCTX0001,lpr385705046400,20140313,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,...,PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [4]:
metadata = metadata[["npr", "scan_date", "patient_age"]]
print(metadata.shape)
metadata.drop_duplicates(inplace=True)
print(metadata.shape)
metadata.head(2)

(3610, 3)
(1805, 3)


,npr,scan_date,patient_age
0,lpr385705046400,20140313,33
2,lpr415675513429,20190201,32


In [5]:
final = pd.merge(data, metadata, how="inner", left_on=["patient_ID", "scan_date"], right_on=["npr", "scan_date"], suffixes=["_l","_r"])
final.head(2)

,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,npr,patient_age
0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,lpr385705046400,33
1,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,lpr415675513429,32


In [6]:
final_data = final.drop(columns=["npr"])

In [8]:
final_data.head()

,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,patient_age
0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,33
1,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,32
2,lpr415675513429,20190320,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,33
3,lpr884752331885,20181116,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,69
4,npr100169878450,20130412,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,/media/andres/T7 Shield1/UCAN_project/2D_proje...,68


In [9]:
final_data.to_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx", index=False)

##### EXTRA CODE

In [1]:
testing = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx")

NameError: name 'pd' is not defined

In [3]:
testing

,Unnamed: 0,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,patient_age
0,0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
1,1,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,32
2,2,lpr415675513429,20190320,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
3,3,lpr884752331885,20181116,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,69
4,4,npr100169878450,20130412,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,1800,npr999470312453,20220201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,72
1801,1801,npr999561275820,20130305,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,82
1802,1802,npr999612862346,20210212,/media/andres/T7 Shield1/UCAN_project

In [14]:
sort_test = testing.sort_values(by="patient_ID")
final_test = sort_test.drop(columns="Unnamed: 0").reset_index(drop=True)
final_test

,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,patient_age
0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
1,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,32
2,lpr415675513429,20190320,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
3,lpr884752331885,20181116,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,69
4,npr100169878450,20130412,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,npr999470312453,20220201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,72
1801,npr999561275820,20130305,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,82
1802,npr999612862346,20210407,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UC

In [16]:
train_data = final_test[:int(final_test.shape[0] * 0.7)].patient_ID.tolist()
train = final_test[final_test.patient_ID.isin(train_data)]
val = final_test[~final_test.patient_ID.isin(train_data)]

In [17]:
train.shape

(1263, 13)

In [18]:
val.shape

(542, 13)

In [22]:
train["patient_age"].nunique()

71

In [12]:
output_path = "/media/andres/T7 Shield1/UCAN_project/collages/"
df_for_collages = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/df_for_collages.xlsx")
df_for_collages.shape

(3646, 14)

In [15]:
df_for_collages["patient_ID"].nunique()

1076

In [16]:
df_for_collages["scan_date"].nunique()

1168

### Updating file names for new system

##### regression

In [2]:
testing = pd.read_excel("/home/ashish/Ashish/UCAN/dataset_for_training_regression.xlsx")

In [3]:
display_full(testing.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                              SUV_MIP                                                                              CT_MIP                                                                              SUV_bone                                                                              CT_bone                                                                              SUV_lean                                                                              CT_lean                                                                              SUV_adipose                                                                              CT_adipose                                                                              SUV_air                                                                              CT_air  patient_age
0           0  lpr385705046400   20140313  /media/andres/T7 Shield1/UCAN_project/collage

In [8]:
testing = testing.replace('/media/andres/T7 Shield1/UCAN_project', '/home/ashish/Ashish/UCAN', regex=True)
display_full(testing.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air  patient_age
0           0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collage

In [9]:
testing.drop(columns='Unnamed: 0', inplace=True)
display_full(testing.head(2))

        patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air  patient_age
0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/201403

In [10]:
testing.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_regression_v1.xlsx", index=False)

##### classification

In [3]:
classification_data = pd.read_excel("/home/ashish/Ashish/UCAN/dataset_for_training_classification.xlsx")

In [4]:
display_full(classification_data.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                              SUV_MIP                                                                              CT_MIP                                                                              SUV_bone                                                                              CT_bone                                                                              SUV_lean                                                                              CT_lean                                                                              SUV_adipose                                                                              CT_adipose                                                                              SUV_air                                                                              CT_air
0           0  lpr385705046400   20140313  /media/andres/T7 Shield1/UCAN_project/collages/lpr38570504

In [5]:
classification_data = classification_data.replace('/media/andres/T7 Shield1/UCAN_project', '/home/ashish/Ashish/UCAN', regex=True)
display_full(classification_data.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air
0           0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr38570504

In [6]:
classification_data.drop(columns='Unnamed: 0', inplace=True)
display_full(classification_data.head(2))

        patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air
0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_bone.n

In [12]:
linked_data = pd.read_excel("/home/ashish/Ashish/UCAN/linked_data_387matches_27nov2023.xlsx")
linked_data = linked_data[["npr", "sex", "diagnosisGroups", "Diagnosis", "Diagnosis 2", "TREATMENT", "indication"]].copy()
display_full(linked_data.head(2))

               npr      sex diagnosisGroups                             Diagnosis Diagnosis 2     TREATMENT indication
0  lpr385705046400  UNKNOWN            HEMA  C81.9 Hodgkins lymfom, ospecificerat         NaN  CHEMOTHERAPY   CURATIVE
1  lpr415675513429  UNKNOWN            HEMA  C81.9 Hodgkins lymfom, ospecificerat         NaN  CHEMOTHERAPY   CURATIVE


In [16]:
classification_data_v1 = pd.merge(classification_data, linked_data, how="inner", left_on=["patient_ID"], right_on=["npr"], suffixes=['','_'])
display_full(classification_data_v1.head(2))

        patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air              npr      sex diagnosisGroups                             Diagnosis Diagnosis 2     TREATMENT indication
0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCA

In [17]:
classification_data_v1.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_classification_v1.xlsx", index=False)

In [20]:
images_with_nan_values = classification_data_v1.copy()
images_with_nan_values['incorrect_projection'] = 'False'

# Loop through each row of the dataset and check if there are any arrays with NaN values in them
for index, row in images_with_nan_values.iterrows():
    arr = np.load(row["SUV_MIP"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_MIP"])
        continue

    arr = np.load(row["SUV_bone"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_bone"])
        continue
        
    arr = np.load(row["SUV_lean"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_lean"])
        continue
        
    arr = np.load(row["SUV_adipose"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_adipose"])
        continue
        
    arr = np.load(row["SUV_air"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["SUV_air"])
        continue
        
    arr = np.load(row["CT_MIP"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_MIP"])
        continue
        
    arr = np.load(row["CT_bone"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_bone"])
        continue
        
    arr = np.load(row["CT_lean"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_lean"])
        continue
        
    arr = np.load(row["CT_adipose"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_adipose"])
        continue
        
    arr = np.load(row["CT_air"])
    if np.isnan(arr).any():
        images_with_nan_values.at[index, 'incorrect_projection'] = 'True'
        print(row["CT_air"])
        continue

display_full(images_with_nan_values[images_with_nan_values['incorrect_projection'] == 'True'])

/home/ashish/Ashish/UCAN/collages/npr272766092791/20201218/SUV_MIP.npy
/home/ashish/Ashish/UCAN/collages/npr925249119000/20181106/SUV_MIP.npy
          patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air              npr   sex diagnosisGroups                Diagnosis Diagnosis 2      TREATMENT indicati

In [27]:
classification_data_v2 = images_with_nan_values[images_with_nan_values['incorrect_projection'] == 'False'].copy()
classification_data_v2 = classification_data_v2[classification_data_v2.sex.isin(["MALE", "FEMALE"])].copy()

classification_data_v2['sex'] = np.where(classification_data_v2['sex']=="MALE", 1, 0)

classification_data_v2.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_classification_v2.xlsx", index=False)

In [26]:
classification_data_v2.npr.nunique()

384